In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Dane bez SMOTE

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Praca inżynierska/Parkinsson disease.csv'
df = pd.read_csv(file_path)
X = df.drop(['name','status'], axis=1)
y = df['status']

SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled,y_resampled = smote.fit_resample(X,y)
print("Rozmiar przed nadprobkowaniem",X.shape)
print("Rozmiar po nadpróbkowaniu",X_resampled.shape)

Rozmiar przed nadprobkowaniem (195, 22)
Rozmiar po nadpróbkowaniu (294, 22)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [ ]:
df1=pd.concat([X_resampled,y_resampled],axis=1)
df1.shape

(294, 23)

In [ ]:
sc_resampled = StandardScaler()
sc_resampled.fit(X_resampled)
X_resampled = sc_resampled.transform(X_resampled)

Podział na zbiory

In [ ]:
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2)

SVC

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='rbf', C=2, gamma=1)

In [ ]:
svc.fit(X_train_resampled, y_train_resampled)

SVC(C=2, gamma=1)

In [ ]:
scores_resampled = cross_val_score(svc, X_resampled, y_resampled, cv=10, scoring='accuracy')
print("Dokładność dla każdego folda walidacji:")
print(scores_resampled)
print("Średnia dokładność:", scores_resampled.mean())
print("Odchylenie standardowe dokładności", scores_resampled.std())
accuracy_svc_resampled = scores_resampled.mean()

Dokładność dla każdego folda walidacji:
[1.         0.9        0.9        1.         1.         1.
 0.96551724 0.96551724 1.         0.93103448]
Średnia dokładność: 0.9662068965517241
Odchylenie standardowe dokładności 0.03964167686399976


Lasy losowe

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)

In [ ]:
rf.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier()

In [ ]:
scores_resampled = cross_val_score(rf, X_resampled, y_resampled, cv=10, scoring='accuracy')
print("\nWyniki dla zbalansowanych danych")
print("Dokładność dla każdego folda walidacji:")
print(scores_resampled)
print("Średnia dokładność:", scores_resampled.mean())
print("Odchylenie standardowe dokładności", scores_resampled.std())
accuracy_rf_resampled = scores_resampled.mean()


Wyniki dla zbalansowanych danych
Dokładność dla każdego folda walidacji:
[0.9        0.86666667 0.9        0.93333333 1.         0.93103448
 0.89655172 0.93103448 0.96551724 0.93103448]
Średnia dokładność: 0.9255172413793102
Odchylenie standardowe dokładności 0.03592317223016844


CatBoost


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, loss_function='Logloss', verbose=False)

In [ ]:
cat.fit(X_train_resampled, y_train_resampled)

In [ ]:
scores_resampled = cross_val_score(cat, X_resampled, y_resampled, cv=10, scoring='accuracy')
print("\nWyniki dla zbalansowanych danych")
print("Dokładność dla każdego folda walidacji:")
print(scores_resampled)
print("Średnia dokładność:", scores_resampled.mean())
print("Odchylenie standardowe dokładności", scores_resampled.std())
accuracy_catboost_resampled = scores_resampled.mean()


Wyniki dla zbalansowanych danych
Dokładność dla każdego folda walidacji:
[0.9        0.9        0.96666667 0.9        1.         1.
 0.89655172 0.96551724 0.96551724 0.96551724]
Średnia dokładność: 0.9459770114942527
Odchylenie standardowe dokładności 0.0402430192024694


Extra Trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_estimators=100)

In [ ]:
et.fit(X_train_resampled, y_train_resampled)

ExtraTreesClassifier()

In [ ]:
scores_resampled = cross_val_score(et, X_resampled, y_resampled, cv=10, scoring='accuracy')
print("\nWyniki dla zbalansowanych danych")
print("Dokładność dla każdego folda walidacji:")
print(scores_resampled)
print("Średnia dokładność:", scores_resampled.mean())
print("Odchylenie standardowe dokładności", scores_resampled.std())
accuracy_catboost_resampled = scores_resampled.mean()


Wyniki dla zbalansowanych danych
Dokładność dla każdego folda walidacji:
[0.93333333 0.96666667 0.86666667 0.96666667 1.         0.93103448
 0.93103448 0.93103448 0.96551724 0.96551724]
Średnia dokładność: 0.9457471264367815
Odchylenie standardowe dokładności 0.03414315529513535
